## QEMU ExecLog

### Notebook Description

In the follwoing notebook we are analyzing the trace of the code excution. The elf file, which is the output of the compilation of our C code, will be executed using QEMU ExecLog tool. 

The execlog tool is from QEMU TCG Pligins (ref: [QEMU TCG Pligins](https://qemu.readthedocs.io/en/latest/devel/tcg-plugins.html)). 

The execlog tool traces executed instructions with memory access. It can be used for debugging and security analysis purposes. 

The below command is used to generate the log file that contains the output logs:

`qemu-system-arm -M versatilepb -m 128M -nographic -kernel test.bin -d plugin -plugin /home/a23balta/qemu/contrib/plugins/libexeclog.so -D log.txt`

Other references:
- [Hello world for bare metal ARM versatile using qemu](https://jasonblog.github.io/note/arm_emulation/hello_world_for_bare_metal_arm_using_qemu.html)
- [Run C program bare metal on ARM Cortex-M3](https://jacobmossberg.se/posts/2018/08/11/run-c-program-bare-metal-on-arm-cortex-m3.html)

### LOAD TRACE DATA

In [3]:
import json, os, re

def profile_logs(log_path):
    inst_stat = {} # inst: count
    b_count = 0
    with open(log_path, "r") as data:
        for line in data:
            match = line.split()
            if match[4] == "pop" or match[4] == "push": # special case for STACK instructions, cycles depends on operand(s)
                m = re.search('{(.+?)}', line)
                if match[4] in inst_stat:
                    inst_stat[match[4]] += len(m.group(1).split(','))
                else:
                    inst_stat[match[4]] = len(m.group(1).split(','))
            if match[4] in inst_stat:
                inst_stat[match[4]] += 1
            else:
                inst_stat[match[4]] = 1

            if match[4] == 'b': b_count += 1
            else: b_count = 0

            if b_count > 5: 
                inst_stat[match[4]] -= 5
                break
    
    return inst_stat

def profile_logs_by_dir(dir_path):
    dir_stat = {}

    for filename in os.listdir(dir_path):
        relative_path = os.path.join(dir_path, filename)

        # checking if it is a file
        if os.path.isfile(relative_path) and relative_path.endswith('.txt'):
            dir_stat[filename.replace('.txt', '')] = profile_logs(relative_path)
    
    return dir_stat

def save_profile_as_json(prof_dict, file_path):
    with open(file_path, 'w') as j_file:
        json.dump(prof_dict, j_file)



In [9]:
all_log_stat = profile_logs_by_dir(dir_path= "/home/a23balta/bare_metal/qemu_logs")
save_profile_as_json(all_log_stat, "results/benchmarks_profile.json")